In [41]:
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics.pairwise import linear_kernel
from bs4 import BeautifulSoup
import re
import requests
import unidecode
import string
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')


In [3]:
hilary = pd.read_csv('data/hilary_scores.csv')

In [50]:
hilary['URL'][1697]
hilary.head()

,Unnamed: 0,Unnamed: 0.1,Comment,URL,Sentiment,Sentiment_b,Subjectivity,Subjectivity_b
0,0,0,I do not see the problem. Japan has always pr...,http://www.nytimes.com/2015/08/01/business/tpp...,0.145714,1,0.338571,1
1,1,1,We persistently run a trade deficit. It is im...,http://www.nytimes.com/2015/08/01/business/tpp...,0.090152,1,0.475758,1
2,2,2,"Again, the world can do, what the US people ca...",http://www.nytimes.com/2015/08/01/business/tpp...,-0.012143,0,0.335714,1
3,3,3,In the photo accompanying this article the rep...,http://www.nytimes.com/2015/08/01/business/tpp...,0.091970,1,0.456288,1
4,4,4,It does not seem like any of the issues under ...,http://www.nytimes.com/2015/08/01/business/tpp...,0.311111,1,0.488889,1


In [51]:
url = 'http://www.nytimes.com/2015/07/16/world/middleeast/obama-diplomacy-iran-nuclear-deal.html'
table = hilary[['Comment', 'URL']][hilary['URL'] == url]
table

,Comment,URL
313,Obama and the Democrats need to remind the pub...,http://www.nytimes.com/2015/07/16/world/middle...
314,Iran realizes that having a bomb is really not...,http://www.nytimes.com/2015/07/16/world/middle...
315,After listening on C-Span to the nuclear exper...,http://www.nytimes.com/2015/07/16/world/middle...
316,The key question is. How effective are the me...,http://www.nytimes.com/2015/07/16/world/middle...
317,"The reality, Iran is and will continue to be a...",http://www.nytimes.com/2015/07/16/world/middle...
318,President Obama has promised Iran that in ten ...,http://www.nytimes.com/2015/07/16/world/middle...
319,I don't see any appeasement in this deal as th...,http://www.nytimes.com/2015/07/16/world/middle...
320,For the first time in all of our history as a ...,http://www.nytimes.com/2015/07/16/world/middle...
321,Two glaring facts seem to be lumped under the ...,http://www.nytimes.com/2015/07/16/world/middle...
322,In the absence of honest skepticism let alone ...,http://www.nytimes.com/2015/07/16/world/middle...


In [6]:
## strip HTML tags
x = hilary['Comment'][1697]
def remove_html_tags(data):
    p = re.compile(r'<[^<]*?>')
    return p.sub('', data)

remove_html_tags(x)

'Why America Is Not the Greatest Country in the World!YouTube:" The Newsroom "America is not the greatest country in the world anymore" 2012 TV series "'

In [36]:
#get body text
html = requests.get(url).content
soup = BeautifulSoup(html, 'html.parser')
body = soup.find_all('p', class_= "story-body-text story-content")

content = '\n'.join([i.text for i in soup.select('p.story-body-text')])
body_text = unidecode.unidecode(content).replace("\n"," ").replace("\'s","").replace("\'t","")
body_text

'WASHINGTON --  On one thing, at least, both sides in the fierce debate over President Obama nuclear accord with Iran agree: He will go down in history because of it. The disagreement relates to how. As a peacemaker or an appeaser? For all of the focus on details like the number of centrifuges or the scope of inspections, the emerging battle represents a larger conflict of visions between Mr. Obama faith in diplomacy as the most rational way to resolve differences and his critics\' deep skepticism over the wisdom of negotiating with what they see as an adversary that cannot be trusted. The Iran agreement, after all, is the culmination of an approach stemming from Mr. Obama first campaign for president, when he vowed to talk with America enemies, a promise that drew scorn not just from Republicans but even from his Democratic rival Hillary Rodham Clinton. Now, with the Iran deal in hand and the reopening of an embassy in Cuba this month, Mr. Obama is realizing that aspiration. This has 

In [32]:
text = 'As a peacemaker or an appeaser?\nFor all of the focus on details like the number of centrifuges or the scope of inspections'
text.replace('\n', ' ')

'As a peacemaker or an appeaser? For all of the focus on details like the number of centrifuges or the scope of inspections'

In [37]:
def tokenize(doc):
    return [snowball.stem(word) for word in word_tokenize(doc.lower())]

In [58]:
tfidfvect = TfidfVectorizer(stop_words='english', tokenizer=tokenize, max_features = 40)


In [60]:
documents = table['Comment']
# vect = CountVectorizer(stop_words='english')
# word_counts = vect.fit_transform(documents)
tfidf_vectorized_body = tfidfvect.fit_transform(body_text).toarray()

cosine_similarities = []

for doc in documents:
    tfidf_vectorized = tfidfvect.fit_transform(doc).toarray()
    cosine_similarities.append(linear_kernel(tfidf_vectorized_body, tfidf_vectorized))

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 38 while Y.shape[1] == 31

In [53]:
for i in range(tfidf_vectorized_body.shape[0]):

7424

In [54]:
tfidf_vectorized_body.shape[1]

38